In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import jax
from interp.model.model_loading import load_model, get_dataset_mean_activations
from interp.tools.grad_modify_query import  ModifierCollectionTreeNode, Query, TargetConf, run_query
from interp.tools.grad_modify_query_items import ItemConf, MulConf
from interp.tools.grad_modify_query_utils import compose_trees
from interp.tools.data_loading import get_val_seqs
import numpy as np
import jax.numpy as jnp
from functools import partial
# model2_name = "GPT2"
model2_name = "apr5_2l"
model_name = "jan5_attn_only_two_layers"
model, params, tokenizer = load_model(model_name, models_dir="/home/ubuntu/interpretability_models_jax")
model_ao, params_ao, tokenizer = load_model(model2_name, models_dir="/home/ubuntu/interpretability_models_jax")

Can't find models at /home/ubuntu/rrfs/interpretability_models_jax//jan5_attn_only_two_layers Cache them locally with `rsync -r ~/rrfs/interpretability_models_jax/ /home/ubuntu/rrfs/interpretability_models_jax//jan5_attn_only_two_layers
loading jan5_attn_only_two_layers from /home/ubuntu/rrfs/interpretability_models_jax//jan5_attn_only_two_layers
Can't find models at /home/ubuntu/rrfs/interpretability_models_jax//GPT2 Cache them locally with `rsync -r ~/rrfs/interpretability_models_jax/ /home/ubuntu/rrfs/interpretability_models_jax//GPT2
loading GPT2 from /home/ubuntu/rrfs/interpretability_models_jax//GPT2


In [4]:
import interp.ui.dataset_search as ds
text = "Yesterday John Von Neumann walked around the park with his dog. John Von Neumann was"
# text1 = "On Saturday, Mr Smith went to the mall."
# text2 = "On Saturday, Mrs Smith went to the mall."
token_ids = tokenizer(text, padding=False, return_tensors="jax")["input_ids"]

In [20]:
result,_=ds.run_fn_on_dataset(partial(ds.diff_between_model_losses,model,params,model_ao,params_ao),n_seqs=1000)
[jnp.std(result),jnp.mean(result.astype(jnp.float64))]

100%|██████████| 250/250 [00:03<00:00, 80.83it/s]


[DeviceArray(2.385, dtype=float16), DeviceArray(-1.0932499, dtype=float32)]

In [ ]:
await ds.show_top_docs_fn(partial(ds.diff_between_model_losses,model,params,model_ao,params_ao),n_seqs=20,model=model)

In [ ]:
from interp.model.gpt_model import inference
from interp.tools.log import KeyIdxs, Idxs, LoggerCache,MutLogCache,SubLogCache,LoggerCacheAll

logger = LoggerCacheAll()
# logger.add(KeyIdxs("blocks.mlp.out_by_neuron", Idxs.all()))
log =  MutLogCache.new(logger=logger)
model.bind(params).blocks[0].mlp(np.random.normal(0,1,(1,10,256)),log=log.sub("blocks").sub("mlp"))
print(log.cache.idxed_cache.keys(),log.cache.cache.keys(),log.cache.cache["blocks.mlp.out_by_neuron"].shape)


In [ ]:

print([[tokenizer.decode([x]),i] for i,x in enumerate(token_ids[0])])
embed_mask = np.zeros((token_ids.shape[1],),dtype=np.bool8)
embed_mask[2] = 1
head_mask = np.zeros((model.num_layers,model.num_heads, token_ids.shape[1]),dtype=np.bool8)
head_mask[0,0,3] = 1
head_mask[1,4,14] = 1
mlp_mask = np.zeros((model.num_layers,model.hidden_size*4,token_ids.shape[1]),dtype=np.bool8)

mlp_mask_tokenless = np.zeros((model.num_layers,model.hidden_size*4), dtype=np.bool8)
head_mask_tokenless = np.zeros((model.num_layers,model.num_heads), dtype=np.bool8)
head_mask_tokenless[0,7] = True
head_mask_tokenless[1,7] = True
# ds.embeds_through_mask_ablation(model,params,token_ids,embed_mask,head_mask,mlp_mask)
ds.between_mask_ablation_unjit(model,params, token_ids,head_mask_tokenless,mlp_mask_tokenless,model_name)

In [ ]:
get_dataset_mean_activations(model_name).keys()

In [ ]:
from interp.ui.attribution_backend import AttributionBackend
backend = AttributionBackend(model, params, tokenizer, text)
vnt_start = backend.startTree(
    {"kind": "logprob", "data": {"seqIdx": 14, "tokString": "Ne", "comparisonTokString": None}}
)
_,_,masks = backend.searchAttributionsFromStart(0.05,True)
print({k:jnp.sum(v) for k,v in masks.items()})
masks_tokenless = {"embeds":np.any(masks["embeds"],axis=-1,keepdims=True),"heads":np.any( masks["heads"],axis=-1,keepdims=True),"mlps":np.any( masks["mlps"],axis=-1,keepdims=True)}
print(masks_tokenless)
ds.embeds_through_mask_ablation(model,params,token_ids,masks["embeds"],masks["heads"],np.expand_dims(masks["mlps"],axis=1))



In [ ]:
import interp.cui as cui
import jax.numpy as jnp
import jax
from interp.ui.very_named_tensor import VeryNamedTensor
# import interp.tensor_makers.atttention_weighted as aw
# import interp.tensor_makers.correct_logprobs as cp
# import interp.tensor_makers.logprobs as mp
await cui.init(port=6789)

In [ ]:
masks_tokenless["embeds"].shape

In [ ]:
await ds.show_top_docs_fn(lambda token_ids: ds.embeds_through_mask_ablation(model,params,token_ids,masks_tokenless["embeds"],masks_tokenless["heads"],masks_tokenless["neurons"]),n_seqs=100,batch_size=1,seq_len=64,model=model)

In [ ]:
from interp.tools.data_loading import get_val_seqs
from interp.model.gpt_model import inference,inference_unjit
from tqdm import tqdm

def collect_mean_activations(model,params, n=11000,batch_size=8,seq_len=512):
    trimmed_n = (n//batch_size)*batch_size
    seqs = get_val_seqs()[:trimmed_n,:seq_len]
    seq_len = seqs.shape[-1]
    
    seqs = seqs.reshape(-1,batch_size,seq_len)
    to_log = [f"blocks.{l}.attention.out_by_head" for l in range(model.num_layers)]
    if model.use_mlp:
        to_log.extend([f"blocks.{l}.mlp.out_by_neuron" for l in range(model.num_layers)])
    to_log=tuple(to_log)
    sums = None
    for seq in tqdm(seqs):
        _,log = inference(model,params,seq,to_log)
        if sums==None:
            sums = {k:jnp.mean(log[k],axis=0) for k in to_log}
        else:
            for k,v in sums.items():
                sums[k] = v+jnp.mean(log[k],axis=0)
    sums = {k:jnp.mean(v/seqs.shape[0],axis=1) for k,v in sums.items()}
    stacked= {"attention":jnp.stack([sums[f"blocks.{l}.attention.out_by_head"] for l in range(model.num_layers)])}
    if model.use_mlp:
        stacked["mlp"]=jnp.stack([sums[f"blocks.{l}.mlp.out_by_neuron"] for l in range(model.num_layers)])
    return stacked
mean_activations = collect_mean_activations(model,params,n=100,batch_size=6)
print([k.shape for k in mean_activations.values()])

In [ ]:
import os
from flax.serialization import msgpack_serialize
def save_all_avg_activations():
    # for model_name in ["jan5_attn_only_two_layers", "jan11_gpt_2l", "dec30_attn_only_two_layers", "GPT2"]:
    for model_name in ["jan11_gpt_2l"]:
        model, params, tokenizer = load_model(model_name)
        mean_activations = collect_mean_activations(model,params,batch_size=1,seq_len=64)
        try:
            os.mkdir(f"/home/ubuntu/interpretability_model_stats/{model_name}")
        except:
            pass
        open(f"/home/ubuntu/interpretability_model_stats/{model_name}/mean_activations","wb").write(msgpack_serialize(mean_activations))
save_all_avg_activations()

In [ ]:

log = LogCacheAll()
model_b(val_seqs[0:1], log=log)

In [ ]:
log['blocks.1.mlp.linear1']

In [ ]:
from copy import deepcopy
A = ds.get_mlp_stats_unjit(model, params, input_ids=val_seqs[1:5])
A.shape

In [ ]:
from dataset_search import *

def get_mlp_grads_unjit(model, params, input_ids_batch,l):

    result = run_query(
        loss_by_token_loggable(model, params, input_ids),
        query=Query(
            targets=[TargetConf("loss.lm_loss", display_name="loss")],
            modifier_collection_tree=ModifierCollectionTreeNode(MulConf(ItemConf(f"blocks.{l}.mlp.linear1"),shape=I[1,1,4*model.hidden_size])),
            use_fwd=False,
        ),
    )
    
    result = result['loss'][:,:,0,0,:]
    return result

dataset_here = get_val_seqs()
batch = dataset_here[:2,:128]
# dataset_batches = dataset_here.reshape(-1, 10, 100)
# print(dataset_batches.shape)
result = get_mlp_grads_unjit(model, params, batch, 0)

In [ ]:
result.shape

In [ ]:
result.shape

In [ ]:
from tqdm import tqdm
from interp.ui.dataset_search import get_mlp_stats

n_seqs = 3000
batch_size = 10
seq_len = 100
k = 5
agg = 'mean'

assert agg in {'mean', 'max'}
dataset_here = get_val_seqs()[: (n_seqs // batch_size) * batch_size, :seq_len]
dataset_batches = dataset_here.reshape(-1, batch_size, seq_len)
result = []
top_k_doc_idx = np.zeros((k, model.num_layers, model.hidden_size*4), dtype=np.int32)
top_k_vals = -np.infty*np.ones((k, model.num_layers, model.hidden_size*4))
for i, batch in tqdm(enumerate(dataset_batches)):
    activations = np.array(get_mlp_stats(model, params, batch), dtype=np.float16)
    if agg == 'mean':
        per_batch_vals = activations.mean(axis=-2)
    elif agg == 'max':
        per_batch_vals = activations.max(axis=-2)
    else:
        assert False,"agg should be in {'mean', 'max'} (and an assertion should have caught it earlier)"

    
    batch_and_top_k_vals = np.concatenate((per_batch_vals, top_k_vals), axis=0)
    bk_top_idx = batch_and_top_k_vals.argsort(axis=0)[-k:]
    top_k_vals = np.take_along_axis(batch_and_top_k_vals, bk_top_idx, axis=0)
    
    new_k_idx, l_idx, n_idx = np.where(bk_top_idx >= batch_size)
    top_k_doc_idx[new_k_idx, l_idx, n_idx] = top_k_doc_idx[bk_top_idx[new_k_idx, l_idx, n_idx] - batch_size, l_idx, n_idx]
    top_k_doc_idx[bk_top_idx < batch_size] = bk_top_idx[bk_top_idx < batch_size] + i*batch_size

top_k_doc_idx, top_k_vals = np.moveaxis(top_k_doc_idx, 0, -1), np.moveaxis(top_k_vals, 0, -1)    
top_k_doc_idx, top_k_vals = np.flip(top_k_doc_idx, axis=-1), np.flip(top_k_vals, axis=-1)


In [ ]:
for i in range(5):
    print(tokenizer.decode(dataset_here[top_k_doc_idx[1,8]][i]))

In [ ]:
from interp.ui.dataset_search import *


def get_mlp_grads_unjit(model, params, input_ids, l):

    result = run_query(
        loss_by_token_loggable(model, params, input_ids),
        query=Query(
            targets=[TargetConf("loss.lm_loss", display_name="loss")],
            modifier_collection_tree=ModifierCollectionTreeNode(MulConf(ItemConf(f"blocks.{l}.mlp.linear1"),shape=[1,1,model.hidden_size * 4])),
            use_fwd=True,
        ),
    )
    print("RESULT SHAPE", result.shape)
    return result

get_mlp_grads_unjit(model, params, input_ids=dataset_batches[1:2], l=1)


In [ ]:
result['loss'].shape

In [ ]:
result['loss'].shape

In [ ]:
# top_k_doc_idx[new_top_k_idx < batch_size] = new_top_k_idx[new_top_k_idx < batch_size] + i*batch_size
# top_k_doc_idx[new_top_k_idx >= batch_size] = 
k_and_doc_idx, layer_idx, neuron_idx = np.where(new_top_k_idx >= batch_size)
k_and_doc_idx-batch_size

In [ ]:

A[A.argsort(axis=0)[-2:]]

In [ ]:
A[0].shape

In [ ]:
out

In [ ]:
print(log.cache['blocks.1.mlp.gelu'].shape)
print(log.cache['blocks.1.mlp.gelu.identity'].shape)


In [ ]:
keys = set(log.cache.keys())
{k for k in keys if ('mlp' in k) and ('blocks.0' in k) and ('norm' not in k)}

In [ ]:
await ds.show_path_contribution_histogram(model,params,((1,5),),10)